## DATA FROM THE WEB

In [ ]:
from bs4 import BeautifulSoup
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as rq
import seaborn as sns
sns.set_context('notebook')

In [121]:
def getParamsRequestValue(i_reportmodel_num, section, year, semester): 
    r = rq.get('http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter', params={'ww_i_reportmodel' : i_reportmodel_num})
    soup = BeautifulSoup(r.text, 'html.parser')
    parameters_map = { 'ww_x_UNITE_ACAD' : section, 'ww_x_PERIODE_ACAD' : year, 'ww_x_UNITE_ACAD' : semester }
    d = {key: next((option.attrs['value'] for option in soup.find('select', {'name': key }) if option.text==value), None) for key, value in parameters_map.items()}

    return d
print(getParamsRequestValue(133685247, 'Informatique', '2016-2017', 'Bachelor Semestre 1'))

{'ww_x_UNITE_ACAD': None, 'ww_x_PERIODE_ACAD': '355925344'}
